In [17]:
import pandas as pd

# Load the datasets
left_dataset_path = '/Users/haoranw/Desktop/5210 final project_Group_8/Data/left_dataset.csv'
right_dataset_path = '/Users/haoranw/Desktop/5210 final project_Group_8/Data/right_dataset.csv'

left_df = pd.read_csv(left_dataset_path)
right_df = pd.read_csv(right_dataset_path)

In [18]:
# delete the columns of "size", "categroy"
left_df.drop(columns=['categories'], inplace=True)
right_df.drop(columns=['size'], inplace=True)

In [19]:
# Change postal_code to zip_code
left_df['postal_code'] = left_df['postal_code'].apply(lambda x: str(int(x)) if pd.notna(x) else '')

left_df.rename(columns={'postal_code': 'zip_code'}, inplace=True)

# For the right dataset, split the 'zip_code' on the hyphen and take the first part
right_df['zip_code'] = right_df['zip_code'].apply(lambda x: x.split('-')[0].strip())

In [20]:
# Trim the zip_code to the first five digits in both datasets, handling None values
left_df['zip_code'] = left_df['zip_code'].apply(lambda x: x[:5] if x and len(x) >= 5 else x)
right_df['zip_code'] = right_df['zip_code'].apply(lambda x: x[:5] if x and len(x) >= 5 else x)

left_df['zip_code'].head(), right_df['zip_code'].head()


(0    63123
 1    19107
 2    18054
 3    37015
 4    63144
 Name: zip_code, dtype: object,
 0    33762
 1    33605
 2    33765
 3    33626
 4    33614
 Name: zip_code, dtype: object)

In [21]:
for column in left_df.columns:
    if pd.api.types.is_numeric_dtype(left_df[column]):
        left_df[column] = left_df[column].astype(str)

for column in right_df.columns:
    if pd.api.types.is_numeric_dtype(right_df[column]):
        right_df[column] = right_df[column].astype(str)



In [22]:
#create block_key
left_df['block_key'] = (left_df['zip_code'].str[:3] + 
                        left_df['state'] + 
                        left_df['name'].str[0].str.upper())

right_df['block_key'] = (right_df['zip_code'].str[:3] + 
                         right_df['state'] + 
                         right_df['name'].str[0].str.upper())

# Ensure the new columns are strings and handle any potential missing values
left_df['block_key'] = left_df['block_key'].astype(str)
right_df['block_key'] = right_df['block_key'].astype(str)

In [23]:
left_df

entity_id                              name  \
0             1                     The UPS Store   
1             2                St Honore Pastries   
2             3          Perkiomen Valley Brewery   
3             4                    Sonic Drive-In   
4             5                   Famous Footwear   
...         ...                               ...   
94580     94581     Adelita Taqueria & Restaurant   
94581     94582  INSPcenter/Thai Clinical Massage   
94582     94583              Wild Birds Unlimited   
94583     94584                 Claire's Boutique   
94584     94585                           Sic Ink   

                                          address          city state  \
0                 87 Grasso Plaza Shopping Center        Affton    MO   
1                                     935 Race St  Philadelphia    PA   
2                                   101 Walnut St    Green Lane    PA   
3                                   615 S Main St  Ashland City    TN   
4      8522 Eager Road, Dierbergs Brentwood Point     Brentwood    MO   
...                                           ...           ...   ...   
94580                               1108 S 9th St  Philadelphia    PA   
94581                 2625 N Meridian St, Unit 50  Indianapolis    IN   
94582                          2813 Bransford Ave     Nashville    TN   
94583                      6020 E 82nd St, Ste 46  Indianapolis    IN   
94584                       238 Apollo Beach Blvd  Apollo beach    FL   

      zip_code block_key  
0        63123    631MOT  
1        19107    191PAS  
2        18054    180PAP  
3        37015    370TNS  
4        63144    631MOF  
...        ...       ...  
94580    19147    191PAA  
94581    46208    462INI  
94582    37204    372TNW  
94583    46250    462INC  
94584    33572    335FLS  

[94585 rows x 7 columns]

In [24]:
right_df

business_id                                  name  \
0               1                 SOURINI PAINTING INC.   
1               2                  WOLFF DOLLA BILL LLC   
2               3     COMPREHENSIVE SURGERY CENTER, LLC   
3               4              FRANK & ADAM APPAREL LLC   
4               5             MORENO PLUS TRANSPORT INC   
...           ...                                   ...   
91786       91787                     DEH TRANSPORT LLC   
91787       91788                           TM INC. LLC   
91788       91789  PASSION TRANSPORTATION SERVICES, LLC   
91789       91790     ROSIE CLEANING SERVICE/COMMERCIAL   
91790       91791                     THE AQUADUCKS LLC   

                            address        city state zip_code block_key  
0                   12800 44th St N  Clearwater    FL    33762    337FLS  
1                   1905 E 19th Ave       Tampa    FL    33605    336FLW  
2      1988 GULF TO BAY BLVD, Ste 1  CLEARWATER    FL    33765    337FLC  
3                  13640 Wright Cir       Tampa    FL    33626    336FLF  
4         8608 Huron Court unite 58       Tampa    FL    33614    336FLM  
...                             ...         ...   ...      ...       ...  
91786               737 E Statue Ct    Franklin    TN    37067    370TND  
91787              4911 Georgia Ave   Nashville    TN    37209    372TNT  
91788                 2005 QUAIL DR   NASHVILLE    TN    37207    372TNP  
91789                  705 N 9TH ST   NASHVILLE    TN    37206    372TNR  
91790                128 Gordon Ter   Nashville    TN    37207    372TNT  

[91791 rows x 7 columns]

In [25]:
import pandas as pd
from unidecode import unidecode
import re

def normalize_text(column):
    column = column.astype(str)
    column = column.str.lower()
    column = column.apply(unidecode)
    column = column.str.replace(r'[^\w\s]', '', regex=True)
    return column


# Normalize business names
left_df['name'] = normalize_text(left_df['name'])
right_df['name'] = normalize_text(right_df['name'])

# Normalize addresses
left_df['address'] = normalize_text(left_df['address'])
right_df['address'] = normalize_text(right_df['address'])

In [26]:
# remove the white space, and Standardize the address 

import pandas as pd
import re

def standardize_address(address):
    # Convert to string in case there are any non-string types
    address = str(address)
    # Remove any leading/trailing whitespace
    address = address.strip()
    # Replace common street suffix abbreviations with a standard
    street_suffixes = {
        'st.': 'Street', 'rd.': 'Road', 'ave.': 'Avenue',
        'dr.': 'Drive', 'ln.': 'Lane', 'blvd.': 'Boulevard'
    }
    for suffix in street_suffixes:
        address = re.sub(r'\b' + suffix + r'\b', street_suffixes[suffix], address, flags=re.IGNORECASE)
    # Standardize directional prefixes/suffixes
    directions = {
        'n ': 'North ', 's ': 'South ', 'e ': 'East ', 'w ': 'West ',
        'ne ': 'Northeast ', 'nw ': 'Northwest ', 'se ': 'Southeast ', 'sw ': 'Southwest '
    }
    for direction in directions:
        address = re.sub(r'\b' + direction, directions[direction], address, flags=re.IGNORECASE)
    # Standardize suite numbers
    address = re.sub(r'\bste\.? ', 'Suite ', address, flags=re.IGNORECASE)
    address = re.sub(r'\bapt\.? ', 'Apartment ', address, flags=re.IGNORECASE)
    # Remove all other special characters except for the hyphen and forward slash
    address = re.sub(r'[^\w\s/-]', '', address)
    # Replace multiple spaces with a single space
    address = re.sub(r'\s+', ' ', address)
    return address

# Apply the function to the address column of your DataFrames
left_df['address'] = left_df['address'].apply(standardize_address)
right_df['address'] = right_df['address'].apply(standardize_address)


In [27]:
# Apply title case to the 'city' column of your DataFrames
left_df['city'] = left_df['city'].str.title()
right_df['city'] = right_df['city'].str.title()


In [28]:
# add combine column

left_df['zip_code'] = left_df['zip_code'].astype(str)
right_df['zip_code'] = right_df['zip_code'].astype(str)

left_df['combined'] = left_df[['name', 'address', 'zip_code']].agg(' '.join, axis=1)
right_df['combined'] = right_df[['name', 'address', 'zip_code']].agg(' '.join, axis=1)


In [29]:
left_df

entity_id                             name  \
0             1                    the ups store   
1             2               st honore pastries   
2             3         perkiomen valley brewery   
3             4                    sonic drivein   
4             5                  famous footwear   
...         ...                              ...   
94580     94581     adelita taqueria  restaurant   
94581     94582  inspcenterthai clinical massage   
94582     94583             wild birds unlimited   
94583     94584                 claires boutique   
94584     94585                          sic ink   

                                         address          city state zip_code  \
0                87 grasso plaza shopping center        Affton    MO    63123   
1                                    935 race st  Philadelphia    PA    19107   
2                                  101 walnut st    Green Lane    PA    18054   
3                              615 South main st  Ashland City    TN    37015   
4      8522 eager road dierbergs brentwood point     Brentwood    MO    63144   
...                                          ...           ...   ...      ...   
94580                          1108 South 9th st  Philadelphia    PA    19147   
94581          2625 North meridian Streetunit 50  Indianapolis    IN    46208   
94582                         2813 bransford ave     Nashville    TN    37204   
94583             6020 East 82nd StreetStreet 46  Indianapolis    IN    46250   
94584                      238 apollo beach blvd  Apollo Beach    FL    33572   

      block_key                                           combined  
0        631MOT  the ups store 87 grasso plaza shopping center ...  
1        191PAS               st honore pastries 935 race st 19107  
2        180PAP       perkiomen valley brewery 101 walnut st 18054  
3        370TNS              sonic drivein 615 South main st 37015  
4        631MOF  famous footwear 8522 eager road dierbergs bren...  
...         ...                                                ...  
94580    191PAA  adelita taqueria  restaurant 1108 South 9th st...  
94581    462INI  inspcenterthai clinical massage 2625 North mer...  
94582    372TNW      wild birds unlimited 2813 bransford ave 37204  
94583    462INC  claires boutique 6020 East 82nd StreetStreet 4...  
94584    335FLS                sic ink 238 apollo beach blvd 33572  

[94585 rows x 8 columns]

In [30]:
right_df

business_id                                 name  \
0               1                 sourini painting inc   
1               2                 wolff dolla bill llc   
2               3     comprehensive surgery center llc   
3               4              frank  adam apparel llc   
4               5            moreno plus transport inc   
...           ...                                  ...   
91786       91787                    deh transport llc   
91787       91788                           tm inc llc   
91788       91789  passion transportation services llc   
91789       91790     rosie cleaning servicecommercial   
91790       91791                    the aquaducks llc   

                                  address        city state zip_code  \
0                      12800 44th Streetn  Clearwater    FL    33762   
1                      1905 East 19th ave       Tampa    FL    33605   
2      1988 gulf to bay BoulevardStreet 1  Clearwater    FL    33765   
3                        13640 wright cir       Tampa    FL    33626   
4               8608 huron court unite 58       Tampa    FL    33614   
...                                   ...         ...   ...      ...   
91786                  737 East statue ct    Franklin    TN    37067   
91787                    4911 georgia ave   Nashville    TN    37209   
91788                       2005 quail dr   Nashville    TN    37207   
91789                    705 North 9th st   Nashville    TN    37206   
91790                      128 gordon ter   Nashville    TN    37207   

      block_key                                           combined  
0        337FLS      sourini painting inc 12800 44th Streetn 33762  
1        336FLW      wolff dolla bill llc 1905 East 19th ave 33605  
2        337FLC  comprehensive surgery center llc 1988 gulf to ...  
3        336FLF     frank  adam apparel llc 13640 wright cir 33626  
4        336FLM  moreno plus transport inc 8608 huron court uni...  
...         ...                                                ...  
91786    370TND         deh transport llc 737 East statue ct 37067  
91787    372TNT                  tm inc llc 4911 georgia ave 37209  
91788    372TNP  passion transportation services llc 2005 quail...  
91789    372TNR  rosie cleaning servicecommercial 705 North 9th...  
91790    372TNT             the aquaducks llc 128 gordon ter 37207  

[91791 rows x 8 columns]

In [31]:
import pandas as pd
import recordlinkage
from recordlinkage.index import Block

# Assuming previous steps are the same as provided (data loading, cleaning, and normalization)

# Using recordlinkage package

# Create an indexer object and use blocking on 'block_key'
indexer = Block('block_key')
pairs = indexer.index(left_df, right_df)

# Initialize the comparison object
compare = recordlinkage.Compare()

# Add comparison criteria. Here compare the 'combined' column using the Jaro-Winkler method.
compare.string('combined', 'combined', method='jarowinkler', threshold=0.8, label='combined_score')

# Compute the comparison vector
features = compare.compute(pairs, left_df, right_df)

# Filter matches based on the score threshold
matches = features[features['combined_score'] > 0.8]

matched_indices = matches.index

# Create a DataFrame from the matches with only necessary columns
results = [(left_id, right_id, score) for (left_id, right_id), score in matches.iterrows()]

# Display or save the results
print(results)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
import pandas as pd
import recordlinkage
from recordlinkage.index import Block

# Assuming previous steps are the same as provided (data loading, cleaning, and normalization)

# Using recordlinkage package

# Create an indexer object and use blocking on 'block_key'
indexer = Block('block_key')
pairs = indexer.index(left_df, right_df)

# Initialize the comparison object
compare = recordlinkage.Compare()

# Add comparison criteria. Here compare the 'combined' column using the Jaro-Winkler method.
compare.string('combined', 'combined', method='jarowinkler', threshold=0.85, label='combined_score')

# Compute the comparison vector
features = compare.compute(pairs, left_df, right_df)

# Filter matches based on the score threshold
matches = features[features['combined_score'] > 0.85]

# Create a DataFrame from the matches with only necessary columns
results = pd.DataFrame({
    'left_id': matches.index.get_level_values(0),
    'right_id': matches.index.get_level_values(1),
    'score': matches['combined_score']
})

# Save the results to a CSV file
results.to_csv('matching_results.csv', index=False)

print("Results have been saved to 'matching_results.csv'")


Results have been saved to 'matching_results.csv'
